# Example 6: Cloud Logging - Datadog Integration

## What You'll Learn

This notebook demonstrates cloud logging with Datadog:

- **DatadogHandler** - Send logs to Datadog
- **Datadog_ServiceConfig** - Configure Datadog integration
- **API Key Management** - Secure configuration
- **Cloud vs Local** - When to use cloud logging
- **Real-World Examples** - Production monitoring scenarios


## Setup - Imports


In [ ]:
import asyncio
import os
from dc_logger.client.Log import LogEntry, LogLevel
from dc_logger.client.base import Logger, HandlerInstance
from dc_logger.services.cloud.datadog import DatadogHandler, Datadog_ServiceConfig

print("All imports successful!")


---
## Part 1: Basic Datadog Configuration

**Note**: To actually send logs to Datadog, you need:
1. A Datadog account
2. A valid API key
3. Optionally, an APP key for additional features

For this example, we'll show the configuration. Replace placeholders with your actual keys.


In [ ]:
print("=" * 70)
print("Basic Datadog Configuration")
print("=" * 70)

# Example configuration - USE YOUR ACTUAL KEYS FOR REAL USAGE
datadog_config = Datadog_ServiceConfig(
    service_name="datadog_logging",
    api_key=os.getenv("DATADOG_API_KEY", "YOUR_API_KEY_HERE"),  # Get from environment
    app_key=os.getenv("DATADOG_APP_KEY", "YOUR_APP_KEY_HERE"),  # Optional
    site="datadoghq.com",  # or us3.datadoghq.com, us5.datadoghq.com, etc.
    service="my_application",  # Your service name in Datadog
    env="production"  # Environment: production, staging, development
)

print("\nDatadog Configuration:")
print(f"  Site: {datadog_config.site}")
print(f"  Service: {datadog_config.service}")
print(f"  Environment: {datadog_config.env}")
print(f"  Intake URL: {datadog_config.derive_intake_url()}")
print(f"  API Key: {'Set' if datadog_config.api_key != 'YOUR_API_KEY_HERE' else 'Not Set'}")


---
## Part 2: Creating Datadog Logger

**Important**: The following cells demonstrate the setup. If you don't have valid Datadog credentials, the actual log sending will fail (which is expected).


In [ ]:
print("=" * 70)
print("Creating Datadog Logger")
print("=" * 70)

# Note: This will work if you have valid credentials
# Otherwise, it will show configuration but not send logs

try:
    # Create Datadog handler
    datadog_handler = DatadogHandler(config=datadog_config)
    
    # Create handler instance
    datadog_handler_instance = HandlerInstance(
        handler=datadog_handler,
        output_type="json"  # Datadog accepts JSON format
    )
    
    # Create logger with Datadog handler
    datadog_logger = Logger(
        app_name="cloud_demo",
        handlers=[datadog_handler_instance]
    )
    
    print("\nDatadog logger created successfully!")
    print("Ready to send logs to Datadog")
    
except Exception as e:
    print(f"\nNote: Could not initialize Datadog handler: {e}")
    print("This is expected if you don't have valid Datadog credentials")
    print("Configuration shown above is for reference")


---
## Part 3: Example Log Entries for Datadog

Here's what log entries would look like when sent to Datadog.


In [ ]:
print("=" * 70)
print("Example Log Entries for Datadog")
print("=" * 70)

# These examples show what you would log if you had valid credentials

examples = [
    {
        "level": "INFO",
        "message": "Application started",
        "kwargs": {
            "extra": {"version": "1.2.3", "environment": "production"}
        }
    },
    {
        "level": "INFO",
        "message": "User authenticated",
        "kwargs": {
            "user_id": "user_alice_123",
            "tenant_id": "tenant_acme",
            "action": "login",
            "extra": {"login_method": "oauth2"}
        }
    },
    {
        "level": "WARNING",
        "message": "High memory usage",
        "kwargs": {
            "extra": {"memory_percent": 85, "threshold": 80}
        }
    },
    {
        "level": "ERROR",
        "message": "Database connection failed",
        "kwargs": {
            "extra": {
                "host": "db.example.com",
                "error": "Connection timeout",
                "retry_count": 3
            }
        }
    }
]

print("\nExample logs that would be sent to Datadog:\n")
for i, ex in enumerate(examples, 1):
    print(f"{i}. [{ex['level']}] {ex['message']}")
    if 'user_id' in ex['kwargs']:
        print(f"   User: {ex['kwargs']['user_id']}")
    if 'action' in ex['kwargs']:
        print(f"   Action: {ex['kwargs']['action']}")
    if 'extra' in ex['kwargs']:
        print(f"   Extra: {ex['kwargs']['extra']}")
    print()


---
## Part 4: Different Datadog Sites

Datadog has different sites depending on your region.


In [ ]:
print("=" * 70)
print("Datadog Sites and Intake URLs")
print("=" * 70)

sites = [
    "datadoghq.com",  # US1
    "us3.datadoghq.com",  # US3
    "us5.datadoghq.com",  # US5
    "datadoghq.eu",  # EU
    "ap1.datadoghq.com",  # AP1
]

print("\nDatadog site intake URLs:\n")
for site in sites:
    url = Datadog_ServiceConfig._derive_intake_url(site)
    print(f"  {site:25} -> {url}")

print("\nNote: Use the site that matches your Datadog account region")


---
## Part 5: Best Practices for Cloud Logging


In [ ]:
print("=" * 70)
print("Best Practices for Cloud Logging")
print("=" * 70)

practices = [
    {
        "title": "1. Secure API Keys",
        "tips": [
            "Store API keys in environment variables",
            "Never commit keys to version control",
            "Use secret management services (AWS Secrets Manager, etc.)",
            "Rotate keys regularly"
        ]
    },
    {
        "title": "2. Use Structured Logging",
        "tips": [
            "Include relevant context (user_id, tenant_id, etc.)",
            "Use consistent field names across your application",
            "Add tags for filtering (env, service, version)",
            "Include trace IDs for distributed tracing"
        ]
    },
    {
        "title": "3. Optimize for Cost",
        "tips": [
            "Log at appropriate levels (don't log DEBUG in production)",
            "Use sampling for high-volume logs",
            "Filter sensitive data before sending",
            "Set up log retention policies"
        ]
    },
    {
        "title": "4. Monitor Your Logging",
        "tips": [
            "Set up alerts for errors and warnings",
            "Monitor log ingestion rates",
            "Check for failed log sends",
            "Review log quality regularly"
        ]
    }
]

for practice in practices:
    print(f"\n{practice['title']}")
    print("-" * 50)
    for tip in practice['tips']:
        print(f"  - {tip}")


---
## Part 6: When to Use Cloud Logging


In [ ]:
print("=" * 70)
print("When to Use Cloud Logging vs File Logging")
print("=" * 70)

comparison = [
    {
        "scenario": "Production Applications",
        "cloud": "Recommended - Centralized monitoring, alerting",
        "file": "Backup option for local debugging"
    },
    {
        "scenario": "Development",
        "cloud": "Optional - Can get expensive",
        "file": "Recommended - Fast, no cost"
    },
    {
        "scenario": "Distributed Systems",
        "cloud": "Essential - Correlate logs across services",
        "file": "Difficult - Logs scattered across machines"
    },
    {
        "scenario": "High Volume Logging",
        "cloud": "Expensive - Consider sampling",
        "file": "Cheap - But management overhead"
    },
    {
        "scenario": "Compliance/Audit",
        "cloud": "Good - Built-in retention and access controls",
        "file": "Manual - Need to implement retention"
    },
    {
        "scenario": "Real-Time Monitoring",
        "cloud": "Excellent - Dashboards and alerts",
        "file": "Limited - Need separate tools"
    }
]

print("\n")
print(f"{'Scenario':<30} {'Cloud Logging':<40} {'File Logging'}")
print("=" * 110)
for item in comparison:
    print(f"{item['scenario']:<30} {item['cloud']:<40} {item['file']}")
    print("-" * 110)


---
## Summary

### What We Learned

1. **DatadogHandler** - Send logs to Datadog cloud service
2. **Datadog_ServiceConfig** - Configure Datadog integration with API keys
3. **Different Sites** - Support for all Datadog regions
4. **Secure Configuration** - Use environment variables for API keys
5. **Best Practices** - Security, cost optimization, monitoring
6. **Use Cases** - When to use cloud vs file logging

### Key Takeaways

- **Centralized Monitoring** - All logs in one place
- **Real-Time Alerts** - Get notified of issues immediately
- **Powerful Search** - Query across all services
- **Cost Awareness** - Monitor usage to control costs
- **Security First** - Never expose API keys

### Configuration Checklist

Before using Datadog logging in production:

- [ ] Get Datadog API key from your Datadog account
- [ ] Store API key in environment variable (not in code)
- [ ] Verify correct site for your region
- [ ] Set appropriate service and environment names
- [ ] Test with a few logs first
- [ ] Set up alerts for log ingestion failures
- [ ] Configure log retention policies

### Next Steps

- **Example 7**: Multiple Handlers - Combine console, file, and cloud logging
  - Log to console for development
  - Log to files for backup
  - Log to Datadog for production monitoring
  - All simultaneously with one logger
